In [ ]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from PIL import ImageFile
from keras.preprocessing import image
from skimage.transform import resize

In [ ]:
# Build the pretrained model with our patched version of layers 
model = MobileNetV2(layers=layers, weights=None, include_top=True , classes=3)

model.summary()

In [ ]:
#Epoch 150/1000
#224/224 [==============================] - 287s 1s/step - loss: 0.0187 - acc: 0.9934 - val_loss: 0.4063 - val_acc: 0.9348

#load weights
model.load_weights("MobileNet_5.weights.150-0.41.my_model")

In [ ]:
height = 224
width = 224

#Paths
train_path = 'Data/Train'
validation_path = 'Data/Validation'
test_path = 'Data/Test'

In [ ]:
def preprocess(x):
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x.astype(np.float32))
    return x

def preprocess2(x):
    return preprocess(x)[0]

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def myIterator(path):
    l = []
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    filesWithDir = [os.path.join(path, file) for file in files]
    #while True:
   
    for i, elt in enumerate(filesWithDir):
        # Open the file
        img = plt.imread(elt)
        #print('Le nom de l image : ' , name_of_image)
        #  le redimensionner
        matrix = resize(img,(height, width,3)) * 255
        t = (matrix, img , elt)
        l.append(t)
        
    return l 

#print(len(list(iterator)))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


model.compile(loss='categorical_crossentropy',metrics=['accuracy'],  optimizer = 'Adam')
image_gen2 = ImageDataGenerator(preprocessing_function = preprocess2)
batch_size_valid = 32
def countfiles(path):
    return sum([len(files) for r,d, files in os.walk(path)])


valid_size = countfiles(validation_path)

validation_batches = image_gen2.flow_from_directory(validation_path ,
                                            target_size=(height, width) ,
                                            classes =['Neige' , 'Non_Neige' ,'indetermine'] ,
                                            batch_size = batch_size_valid)

print(model.evaluate_generator(validation_batches, steps=valid_size//32))
print(model.metrics_names)

In [ ]:
#Ploting the first 10 images of a test batch 
imgs_val , labels_val = next(validation_batches)
plt.imshow(imgs_val[4])
print('img_val mean' , imgs_val[4].mean())
print('img_val max' , imgs_val[4].max())
print('img_val max' , imgs_val[4].min())

In [ ]:
#MyIterator of snowy images
data_validation_snowy_images =  myIterator(validation_path+ '/Neige')
label_validation_snowy_images = np.array([1. , 0. , 0.])

#MyIterator of not snowy images
data_validation_not_snowy_images =  myIterator(validation_path+ '/Non_Neige')
label_validation_not_snowy_images = np.array([0. , 1. , 0.])

#MyIterator of undetermined images 
data_validation_undetermined_images =  myIterator(validation_path+ '/indetermine')
label_validation_undetermined_images = np.array([0. , 0. , 1.])

#Put all the predictions in one list
predictions = []
#trois dossiers
for i in range(3):
    if (i == 0):
        true_label = label_validation_snowy_images
        true_data = data_validation_snowy_images
    elif(i == 1):
        true_label = label_validation_not_snowy_images
        true_data = data_validation_not_snowy_images  
    else:
        true_label = label_validation_undetermined_images
        true_data = data_validation_undetermined_images
   # tuple_data = (np matrix resized, img , nom_image)
    for tuple_data in true_data:
        data = tuple_data[0]
        #print(data.dtype)
        #print('Min: %.3f, Max: %.3f' % (data.min(), data.max()))
        data_exp = preprocess(data)
        #print(data_exp.dtype)
        #print('Min: %.3f, Max: %.3f' % (data_exp.min(), data_exp.max()))
        prediction = model.predict(data_exp)
        pred_label = prediction[0]
        #for each image return true_label , pred_label , the image and the name of the image
        elt = true_label , pred_label , tuple_data[1] , tuple_data[2] , tuple_data[0]
        predictions.append(elt)

#print(predictions[0])

In [ ]:
##print out the images that were well predicted
print("Images prédites avec certitude")
count = 0
for elt in predictions:
    true_label , pred_label , image , name_of_image , img = elt
    for i in range(3):
        if (true_label[i] ==1.):
            if (pred_label[i] >= 0.5):
                print(true_label)
                print(pred_label)
                print('Le nom de l image : ' , name_of_image)
                plt.imshow(image)
                plt.show()
                count += 1
                break
print('count = ' + str(count))

In [ ]:
print("Images prédites avec incertitude")
count = 0
for elt in predictions:
    true_label , pred_label , image , name_of_image , img = elt
    for i in range(3):
        if (true_label[i] ==1.):
            if ((all(pred_label[i] >= a for a in pred_label)) and (pred_label[i] < 0.5)) :
                print(true_label)
                print(pred_label)
                print('Le nom de l image : ' , name_of_image)
                plt.imshow(image)
                plt.show()
                count += 1
                break
                
print('count = ' + str(count))

In [ ]:
##print out the images that were not well predicted
print("Images mal prédites")
count = 0

for elt in predictions:
    true_label , pred_label , image , name_of_image , img  = elt
    
    pred_class = np.argmax(pred_label)
    true_class = np.argmax(true_label)
    
    if pred_class != true_class:
        print(true_label)
        print(pred_label)
        print('Le nom de l image : ' , name_of_image)
        plt.imshow(image)
        plt.show()
        count += 1

print('count = ' + str(count))

In [ ]:
import shutil

print('classify all of the images in the dataset')
#classify all of the images in the dataset
DataSet_path = 'DataSet/Images'

#All files in the data set
files = [f for f in os.listdir(DataSet_path) if os.path.isfile(os.path.join(DataSet_path, f))]
filesWithDir = [os.path.join(DataSet_path, file) for file in files]

#Transform each file into a numpy array in order to classify it
for elt in filesWithDir:
        # Open the file
        img = plt.imread(elt)
        matrix = resize(img,(height, width,3)) * 255
        #process the data 
        data_exp = preprocess(matrix)
        prediction = model.predict(data_exp)
        pred_label = prediction[0]
        #get the name of the file
        words = elt.split('/')
        FileName = words[-1]
        #Move the file to the right directory  
        if (all(pred_label[0] >= a for a in pred_label)):
            destination = 'DataSet/Decision_Neige/'+FileName
            shutil.move(elt, destination)
            
        elif(all(pred_label[1] >= a for a in pred_label)):
            destination = 'DataSet/Decison_Non_Neige/'+FileName
            shutil.move(elt, destination)
            
        else:
            destination = 'DataSet/Decision_None/'+FileName
            shutil.move(elt, destination)
            
